In [ ]:
from pathlib import Path
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary
from torchvision import datasets, transforms
import time
import pandas as pd

In [ ]:
# 1. Configuration and Data Loading
BATCH_SIZE = 64

data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
class SingleImageFolder(torch.utils.data.Dataset):
    def __init__(self, root, transform=None, extensions=('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
        self.root = Path(root)
        self.transform = transform
        self.extensions = extensions

        # Collect all image files with given extensions
        self.paths = []
        for ext in extensions:
            self.paths.extend(self.root.glob(f'*{ext}'))
        self.paths = sorted(self.paths)  # ensure consistent ordering

        # Extract class names from file stems
        self.classes = sorted({p.stem for p in self.paths})
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_path = self.paths[idx]
        try:
            img = Image.open(img_path).convert('RGB')   # ensure RGB for consistency
        except Exception as e:
            raise RuntimeError(f"Failed to load image at {img_path}: {e}")

        # Get class label from filename stem
        class_name = img_path.stem
        label = self.class_to_idx[class_name]

        if self.transform:
            img = self.transform(img)

        return img, label

In [ ]:
train_data = datasets.ImageFolder(
    root='./data/asl_alphabet_train',
    transform=data_transform
)
test_data = SingleImageFolder(
    root="./data/asl_alphabet_test",
    transform=data_transform
)

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=4)
test_loader = DataLoader(dataset=test_data,
                             shuffle=True, num_workers=0)

In [ ]:
# Use GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# 2. Define the Neural Network Model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # 200 200
        self.pool0 = nn.MaxPool2d(kernel_size=4) # 50 50

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3)  #48 48
        self.pool1 = nn.MaxPool2d(kernel_size=2)  # 24 24

        self.conv2 = nn.Conv2d(in_channels=2, out_channels=4, kernel_size=3)  #22 22
        self.pool2 = nn.AdaptiveAvgPool2d(output_size=(3, 3))
        # self.pool2 = nn.MaxPool2d(kernel_size=3)

        self.fc3 = nn.Linear(4 * 3 * 3, 29)

    def forward(self, x):
        x = self.pool0(x)

        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = x.view(-1, 4 * 3 * 3)
        x = self.fc3(x)
        # Apply log softmax for the final output (often used with NLLLoss, which is part of CrossEntropyLoss)
        return F.log_softmax(x, dim=1)


model = SimpleNN().to(device)

In [ ]:
# 3. Define Loss and Optimizer
criterion = nn.NLLLoss()  # Negative Log-Likelihood Loss (suitable for log_softmax output)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
summary(model)

In [ ]:
from datetime import datetime

writer = SummaryWriter(f'runs/init_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}')

In [ ]:
def format_time(seconds):
    """Convert seconds to HH:MM:SS format."""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"

In [ ]:
# 4. Training Loop
NUM_EPOCHS = 5

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0
    epoch_start = time.time()
    total_batches = len(train_loader)

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if batch_idx % 100 == 0:
            elapsed = time.time() - epoch_start
            batches_done = batch_idx + 1
            # Avoid division by zero
            avg_time_per_batch = elapsed / batches_done
            remaining_batches = total_batches - batches_done
            remaining = avg_time_per_batch * remaining_batches

            elapsed_str = format_time(elapsed)
            remaining_str = format_time(max(remaining, 0))

            print(f'Train Epoch: {epoch + 1}/{NUM_EPOCHS} '
                  f'[{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / total_batches:.0f}%)]\t'
                  f'Loss: {loss.item():.6f}\t'
                  f'Elapsed: {elapsed_str}\t'
                  f'Remaining: {remaining_str}')

    avg_train_loss = running_loss / total_batches
    writer.add_scalar('Loss/Train', avg_train_loss, epoch)
    epoch_time = time.time() - epoch_start
    print(f'Epoch {epoch+1} completed in {format_time(epoch_time)}')

writer.flush()
writer.close()

In [ ]:
model.eval()

In [ ]:
model.cpu()

In [ ]:
# tensorboard --logdir=runs

In [ ]:
# 5. Testing the Model
loss_items = {}
test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        loss_items[test_data.classes[target]] = [test_loss]

test_loss /= len(test_loader.dataset)
accuracy = 100. * correct / len(test_loader.dataset)

print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
      f'({accuracy:.2f}%)\n')
loss_df = pd.DataFrame(data=loss_items)

In [ ]:
loss_df.T.sort_index().plot(kind="bar")

In [ ]:
# Save the model (optional)
torch.save(model.state_dict(), "mnist_simple_nn.pth")

In [ ]:
def export_onnx(model, onnx_path="dynamic_cnn.onnx"):
    dummy = torch.randn(1, 1, 256, 256)

    with torch.inference_mode():
        torch.onnx.export(
            model,
            dummy,
            onnx_path,
            opset_version=18,
            dynamo=False,
            export_params=True,
            do_constant_folding=True,
            input_names=["input"],
            output_names=["output"],
            dynamic_axes={
                "input": {0: "batch", 2: "height", 3: "width"},
                "output": {0: "batch"},
            },
        )

In [ ]:
onnx_path = "model.onnx"
export_onnx(model, onnx_path)

In [ ]:
import onnx

onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)  # Check for model correctness

print("Model inputs:")
for input in onnx_model.graph.input:
    print(f"Input name: {input.name}")
    print(
        f"Input shape: {[(d.dim_value if d.HasField('dim_value') else d.dim_param) for d in input.type.tensor_type.shape.dim]}")

In [ ]:
# Take a look at model graph
# https://netron.app/